## Linkareer crawling

### [1] 크롤링
[1] main url : https://linkareer.com/cover-letter/search?page=1&sort=PASSED_AT&tab=all
* sort = 최신순
* 한 페이지 당 20개, 총 자소서 14989

In [ ]:
* 중간에 div 선분 있음 (짝수) / 그러므로 홀수 번호만 가져와야 함!
//*[@id="__next"]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[1]/div[1]/a/div/p[1]
//*[@id="__next"]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[1]/div[1]/a
//*[@id="__next"]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[3]/div[1]/a
//*[@id="__next"]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[5]/div[1]/a
....
//*[@id="__next"]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[39]/div[1]/a

In [ ]:
* 아래 내용들 크롤링 

현대엔지니어링 / 자산관리 자산설비 / 2021 상반기 : //*[@id="__next"]/div[1]/div[1]/div/div/div[3]/div[1]/div/div/div[1]/h1
한양대 ERICA / 기계공학과 / 학점 3.78/4.5 /  : //*[@id="__next"]/div[1]/div[1]/div/div/div[3]/div[1]/div/div/div[2]/p
내용 : //*[@id="coverLetterContent"]/main
    

* 전처리 해서 지금 못가져감 일단 다 저장한 다음 한번에 전처리 처리

In [1]:
from selenium import webdriver
    
import requests
import re
from multiprocessing import Process
import json
import os
from time import sleep

import csv
import platform

import traceback

import time

In [5]:
class Writer(object):
    def __init__(self, num, edit):
        self.file = None
        self.initializer_file(edit)
        self.csv_writer = csv.writer(self.file)
        if num == 1:
            self.csv_writer.writerow(['CompanyName', 'CompanyCareer', 'CompanyDate', 'UserInfos', 'Score', 'Content', 'URL'])
        else:
            self.csv_writer.writerow(['Company', 'UserInfos', 'Content', 'URL'])
    
    def initializer_file(self, edit):
        output_path = 'G:\내 드라이브\web_crawling'

        file_name = f'{output_path}/linkareer'+'.csv'

        if os.path.isfile(file_name):
            print("@@@@@@@ this file is remove and restart crawling! @@@@@@@")
        
        user_os = str(platform.system())
        if user_os == "Windows":
            if not edit:
                self.file = open(file_name, 'w', encoding='utf-8', newline='')
            else:
                self.file = open(file_name, 'a', encoding='utf-8', newline='')
        
    def write_row(self, row):
        self.csv_writer.writerow(row)
    
    def close(self):
        self.file.close()

In [6]:
class Linkareer(object):
    def __init__(self):
        self.main_url = f'https://linkareer.com/cover-letter/search?page='
        self.driver = webdriver.Chrome('C:/Users/msi/Downloads/chromedriver_win32/chromedriver.exe')
        self.links = []
        self.error_urls = []
        
    def get_links(self): # get 각 페이지에 있는 list들
        for page in range(1, 495): # 최대 494page임
            self.driver.get(self.main_url+str(page)+'&sort=PASSED_AT&tab=all')
            for i in range(1, 40, 2): # 한 페이지당 href 최대 개수는 20개
                self.links.append(self.driver.find_element_by_xpath(f'//*[@id="__next"]/div[1]/div/div[3]/div[2]/div/div[3]/div[1]/div[{i}]/div[1]/a').get_attribute("href"))  
                if page == 494 and i == 37: return
        return 
    
    def ErrorLog(self, url, error: str): 
        with open("Log.txt", "w") as f:
            f.write(f"[{url}] - {error}\n")
        
    def crawling1(self, check):
        writer = Writer(1, check)
        
        for link in self.links:
            self.driver.get(link)
            print("@@@ current url : ", self.driver.current_url)
            
            title = self.driver.find_element_by_xpath('//*[@id="__next"]/div[1]/div[1]/div/div/div[3]/div[1]/div/div/div[1]/h1').text
            title = title.split("/")
            if len(title) < 3: title += []*(3-len(title))
            print("@@@@ title : ", [t.strip() for t in title])
            
            contents = self.driver.find_element_by_xpath('//*[@id="coverLetterContent"]/main').text
            #print("@@@@ content : ", contents[:15])

            try:
                if '학점' in user_info:
                    user_info = self.driver.find_element_by_xpath('//*[@id="__next"]/div[1]/div[1]/div/div/div[3]/div[1]/div/div/div[2]/p').text
                    score = re.search('학점(.?)\d*(\.?\d*)(./)\d*(\.?\d*)(.)/', user_info)
                    if not score:
                        score = re.search('학점(.?)\d*(\.?\d*)', user_info).group()[0]
                    
                    user_info = user_info.replace(score, '')
                    score = score[2:].split('/')[0]
                else: score = ''
                print("@@@@ user info : ", score, user_info)
                
                # CSV파일에 작성
                writer.write_row([title[0], title[1], title[2], user_info, score, contents, link])

                del title
                del user_info
            except: # user info 없는 경우
                # 에러 났으면, 해당 url이랑 log 적고 넘기기
                err = traceback.format_exc()
                self.ErrorLog(link, str(err))
                self.error_urls.append(link)
                continue
            self.driver.implicitly_wait(0.1)
        writer.close()
    
    def crawling2(self, check):
        writer = Writer(2, check)
        
        for link in self.links:
            self.driver.get(link)
            print("@@@ current url : ", self.driver.current_url)
            
            try:
                title = self.driver.find_element_by_xpath('//*[@id="__next"]/div[1]/div[1]/div/div/div[3]/div[1]/div/div/div[1]/h1').text
                user_info = self.driver.find_element_by_xpath('//*[@id="__next"]/div[1]/div[1]/div/div/div[3]/div[1]/div/div/div[2]/p').text
                contents = self.driver.find_element_by_xpath('//*[@id="coverLetterContent"]/main').text
                writer.write_row([title, user_info, contents, link])
                del title
                del user_info
            except: # user info 없는 경우
                title = self.driver.find_element_by_xpath('//*[@id="__next"]/div[1]/div[1]/div/div/div[3]/div[1]/div/div/div[1]/h1').text
                contents = self.driver.find_element_by_xpath('//*[@id="coverLetterContent"]/main').text
                writer.write_row([title, '', contents, link])
                self.error_urls.append(link)
                continue
            self.driver.implicitly_wait(0.1)
        writer.close()
        
    def main(self):
        self.get_links()
        print("##### Total links : ", len(self.links))
        
        self.crawling()

In [50]:
links

['https://linkareer.com/cover-letter/25156?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/24708?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/25424?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/24965?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/25713?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/26490?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/26420?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/24910?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/25428?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/26507?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/25208?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/24867?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/cover-letter/26470?page=2&sort=PASSED_AT&tab=all',
 'https://linkareer.com/c

In [7]:
# 리스트 확인 작업
Crawler = Linkareer()
Crawler.get_links()

In [8]:
urls = Crawler.links[:]
len(Crawler.links)

9879

In [9]:
Crawler.crawling2(False)

@@@@@@@ this file is remove and restart crawling! @@@@@@@
@@@ current url :  https://linkareer.com/cover-letter/25204?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/cover-letter/25579?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/cover-letter/28660?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/cover-letter/28662?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/cover-letter/28755?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/cover-letter/28753?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/cover-letter/28762?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/cover-letter/28746?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/cover-letter/28665?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/cover-letter/28761?page=1&sort=PASSED_AT&tab=all
@@@ current url :  https://linkareer.com/c

In [10]:
error_urls = Crawler.error_urls[:]
len(Crawler.error_urls)

0

In [ ]:
import pandas as pd

df = DataFrame (urls, columns=['URLS'])
df.to_csv('./Linkareer_URLS.csv', mode = 'w', index=False)

### [2] 내용 확인

In [17]:
import pandas as pd

df = pd.read_csv('G:\내 드라이브\web_crawling\linkareer.csv')

df.isnull().sum()

CompanyName        0
CompanyCareer      0
CompanyDate        0
UserInfos          0
Score            407
Content            0
URL                0
dtype: int64

In [23]:
len(df)

9023

In [21]:
df[df['Score'].isnull() == True]

,CompanyName,CompanyCareer,CompanyDate,UserInfos,Score,Content,URL
170,삼성엔지니어링,플랜트 설계,2020 하반기,"서울년제 / 전기공학과 / 학점.31/4.5 / 토익: 845, 오픽: IH / 사...",NaN,1. 삼성엔지니어링을 지원한 이유와 입사 후 회사에서 이루고 싶은 꿈을 기술하십시오...,https://linkareer.com/cover-letter/25487?page=...
198,한미글로벌건축사사무소,사업관리 전기,2020 하반기,"서울년제 / 전기공학과 / 학점.31/4.5 / 토익: 845, 오픽: IH / 사...",NaN,"1. 지원분야와 관련하여 귀하가 가진 역량(지식, 기술, 열정, 경력)에 대해 기술...",https://linkareer.com/cover-letter/25479?page=...
214,한국동서발전,발전전기,2020 하반기,"서울년제 / 전기공학과 / 학점.31/4.5 / 토익: 845, 오픽: IH / 사...",NaN,1. 귀하가 동서발전에 지원하게 된 계기는 무엇입니까?\n가) 우리 회사에 관심과 ...,https://linkareer.com/cover-letter/25449?page=...
257,효성중공업,건설 전기시공,2020 하반기,"서울년제 / 전기공학과 / 학점.31/4.5 / 토익: 845, 오픽: IH / 사...",NaN,1. 지원 직무 관련 자신이 갖춘 역량에 대하여 구체적으로 기술하여 주십시오.(수강...,https://linkareer.com/cover-letter/25450?page=...
282,서울교통공사,사무,2020 하반기,수도권년제 / 체육학과 / 학점 3.31 /.5 / 토익: 600 / 기타: MOS...,NaN,1. 자신이 왜 서울교통공사의 인재상에 부합되는 인물이라고 생각하는지 구체적으로 기...,https://linkareer.com/cover-letter/25367?page=...
...,...,...,...,...,...,...,...
8863,현대모비스,연구개발,2015 하반기,서울년제 / 컴퓨터공학 / 학점 3.0 / 토익 725 / 특허 유니버시아드 수상,NaN,현대모비스 해당 직무 분야에 지원하게 된 이유를 기재해 주시기 바랍니다. (700자...,https://linkareer.com/cover-letter/13365?page=...
8885,한국후지제록스,영업,2015 상반기,"지방년제 / 상경계열 / 외국계기업 인턴, KT&G 상상 발렌티어, 상상프렌즈 대외활동",NaN,<한국후지제록스 자기소개서 항목>\n\n성장과정 및 자기소개\n\n[시련을 통해 배...,https://linkareer.com/cover-letter/13230?page=...
8918,LG유플러스,네트워크,2015 상반기,서울권년제 / 컴공 / 학점 3.5 / 앱 공모전 수상,NaN,■ ‘회사 및 지원 직무에 대한 지원동기’와 ‘해당 직무를 잘 수행하기 위해 어떤 ...,https://linkareer.com/cover-letter/13151?page=...
8988,동아지질,토목시공,2015 상반기,서울년제 / 토목 / 학점 3.87 / 토익 830,NaN,성장과정 \n\n‘검토의 습관’\n학창시절 100% 완벽하게 준비하지 못하는 성격을...,https://linkareer.com/cover-letter/20228?page=...
